# Ejercicio 1

Gi: guste el programa i, J: es joven, V: es viejo

P(J|G1,G3,~G2,~G4) (1)= P(G1,G3,~G2,~G4|J).P(J) / P(G1,G3,~G2,~G4)
(2)= P(G1|J).P(G2|J).P(~G3|J).P(~G4|J).P(J) / P(G1).P(G2).P(~G3).P(~G4) = 0.18
(1): Teorema de bayes, (2): Asumimos independencia de sucesos

P(V|G1,G3,~G2,~G4) = 1 - P(J|G1,G3,~G2,~G4) = 0.82

# Ejercicio 2

Importamos las librerias

In [ ]:
import pandas as pd
from TP1.src.Ej2 import divide_by_nationality
from TP1.src.NaiveBayes import NaiveBayes

Leemos los datos y los dividimos por nacionalidad

In [ ]:
data = pd.read_excel('../PreferenciasBritanicos.xlsx', sheet_name='Hoja1', )

britanicos = divide_by_nationality(data, 'I')
escocesas = divide_by_nationality(data, 'E')

datasets = [britanicos, escocesas]

Entrenamos Bayes

In [ ]:
nb = NaiveBayes()
nb.train(datasets)

Buscamos la nacionalidad con las caracteristicas solicitadas

In [ ]:
category = nb.calculate_category([1, 0, 1, 1, 0])

print("Es de Nacionalidad :" + str(datasets[category]['Nacionalidad'].values[0]))

# Ejercicio 3

Importamos el dataset y librerias.

In [1]:
from TP1.src.Ej3 import add_columns_from_text, get_categories, separate_in_categories, confusion_matrix, get_metrics, \
    invert_dict, roc_curve
import pandas as pd
from TP1.src.NaiveBayes import NaiveBayes

df = pd.read_csv('../dataset/Noticias_argentinas.csv', nrows=1000)

df = df[['titular', 'categoria']]

Agregamos una columna por palabra que aparece en algun titual y para cada fila indicamos con 1 si pertenece al titular y con 0 si no pertenece.

In [2]:

df = add_columns_from_text(df)

KeyboardInterrupt: 

Obtenemos las 4 categorias que vamos a clasificar.

In [ ]:
categories = get_categories(df)[0:4]

Separamos el dataset en training y test data.

In [ ]:
train = df.sample(frac=0.8)
test = df.drop(train.index)

Particionamos los dataframes en categorias y luego eliminamos las columnas innecesarias.

In [ ]:
train_datasets = separate_in_categories(train, categories)
test_datasets = separate_in_categories(test, categories)

train_datasets = [df.drop(columns=['titular', 'categoria']) for df in train_datasets]
test_datasets = [df.drop(columns=['titular', 'categoria']) for df in test_datasets]

Creamos una instancia de nuestra implementación de Naive Bayes y entrenamos con nuestro dataset de entrenamiento.

In [ ]:
nb = NaiveBayes()
nb.train(train_datasets)

Generamos la matriz de confusión utilizando el dataset de prueba.

In [ ]:
res = confusion_matrix(nb, test_datasets)
res_dic = {categories[i]: {categories[j]: res[i][j] for j in range(len(res[i]))} for i in range(len(res))}

print(pd.DataFrame.from_dict(res_dic))

Calculamos métricas de evaluación.

In [ ]:
metrics_dic = get_metrics(res, categories)
print(pd.DataFrame.from_dict(invert_dict(metrics_dic)))

Generamos un espacio ROC para graficar los resultados para cada categoria.

In [ ]:
roc_curve(metrics_dic)

# Ejercicio 4

Importamos las librerias

In [ ]:
from TP1.src.BayesNetwork import BayesNetwork
import pandas as pd

Leemos los datos y los discretizamos

In [ ]:
data = pd.read_csv('../dataset/binary.csv')

data['gre'] = (data['gre'] >= 500).astype(int)
data['gpa'] = (data['gpa'] >= 3).astype(int)

Creamos la red y calculamos las probabilidades

In [ ]:
bn = BayesNetwork(data, {'rank': [], 'gre': ['rank'], 'gpa': ['rank'], 'admit': ['gre', 'gpa', 'rank']}, {'rank': [1, 2, 3, 4], 'gre': [0, 1], 'gpa': [0, 1], 'admit': [0, 1]})

print("P(admit = 0 | rank = 1) = " + str(bn.conditional_probability({'admit': 0}, {'rank': 1})))
print("P(admit = 1 | rank = 2, gre = 0, gpa = 1) = " + str(bn.conditional_probability({'admit': 1}, {'rank': 2, 'gre': 0, 'gpa': 1})))